In [6]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AutoTokenizer, AutoModelForSequenceClassification
from torch import nn
from tqdm import tqdm

In [7]:
codeBERT = "microsoft/codebert-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained(codeBERT)

In [35]:
prompt_tokens = tokenizer.tokenize("Given the following C code, determine if it has an out of bounds read vulnerability (CWE-125). If there is a vulnerability, output '1'. If there is no vulnerability, output '0'.")
code_tokens = tokenizer.tokenize("int main() { int c = 2; int arr[] = {1}; printf(\"%d\", arr[c]);}")
inputs = tokenizer.encode_plus("".join(prompt_tokens), "".join(code_tokens), add_special_tokens=True, padding=True, truncation=True, return_tensors="pt")
classification_model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
classification_model.to(device)
# i haven't done any fine tuning, this will be done and specific to each CVE and hopefully improve results
for input_data in tqdm([inputs], desc="Processing batch", unit="example"):
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = classification_model(input_ids=input_ids, attention_mask=attention_mask)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Processing batch: 100%|██████████| 1/1 [00:00<00:00,  6.08example/s]


In [36]:
logits = outputs.logits
print(logits)
predicted_class = torch.argmax(logits, dim=1).item()
print(predicted_class) 

tensor([[ 0.0828, -0.0939]])
0
